In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras

In [2]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model

In [4]:
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [5]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [6]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [7]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator)

Epoch 1/10
521/521 [==============================] - 3261s 6s/step - loss: 4.8162 - accuracy: 0.0090 - val_loss: 4.7871 - val_accuracy: 0.0097
Epoch 2/10
521/521 [==============================] - 3674s 7s/step - loss: 4.7908 - accuracy: 0.0107 - val_loss: 4.7868 - val_accuracy: 0.0107
Epoch 3/10
153/521 [=======>......................] - ETA: 35:14 - loss: 4.7852 - accuracy: 0.0125

In [ ]:
model.save('sanford_dog_xception.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')